In [6]:
import simpy
import numpy as np

# Queuing System Simulation processes class
class QueuingSystemSimulation:
    def __init__(self, n, m, lambd, mu, v, env):
        # queuing system params
        self.n = n
        self.m = m
        self.lambd = lambd
        self.mu = mu
        self.v = v
        # empirical values lists
        self.L_queuing_system_list = []
        self.L_queue_list = []
        self.t_queuing_system_list = []
        self.t_queue_list = []
        # enviromental variables for simpy simulation
        self.env = env
        self.loader = simpy.Resource(env, n)

    def run(self):
        while True:
            yield self.env.timeout(np.random.exponential(1/self.lambd))
            self.env.process(self.make_request())

    def make_request(self):
        queque_len_before = len(self.loader.queue)
        n_busy = self.loader.count

        with self.loader.request() as request:
            self.L_queue_list.append(queque_len_before)
            self.L_queuing_system_list.append(queque_len_before + n_busy)
            if len(self.loader.queue) <= self.m:
                arrival_time = self.env.now

                waiting_process = self.env.process(self.waiting_in_queue())
                result = yield request | waiting_process    

                self.t_queue_list.append(self.env.now - arrival_time)
                if request in result:
                    yield self.env.process(self.request_processing())
                    self.t_queuing_system_list.append(self.env.now - arrival_time)
                else:
                    self.t_queuing_system_list.append(self.env.now - arrival_time)
            else:
                self.t_queue_list.append(0)
                self.t_queuing_system_list.append(0)

    def request_processing(self):
        yield self.env.timeout(np.random.exponential(1/self.mu))

    def waiting_in_queue(self):
        yield self.env.timeout(np.random.exponential(1/self.v))

In [7]:
def queuing_system_simulate(n, m, lambd, mu, v, test_time):
    env = simpy.Environment()
    qs = QueuingSystemSimulation(n, m, lambd, mu, v, env)
    env.process(qs.run())
    env.run(until=test_time)
    return qs.t_queuing_system_list, qs.L_queue_list, qs.t_queue_list, qs.L_queuing_system_list

t_queuing_system_list, L_queue_list, t_queue_list, L_queuing_system_list = queuing_system_simulate(3, 3, 2, 1, 1, 10000)
print(np.average(t_queuing_system_list), np.average(L_queue_list),
      np.average(t_queue_list), np.average(L_queuing_system_list))

kek:  0.5699438347274715
kek:  0.08318906592405284
kek:  0.5309412780321381
kek:  0.009697300791472685
kek:  0.137762927922239
kek:  0.02798570356300445
kek:  0.6327038494957975
kek:  0.8068767363345586
kek:  0.03321229337045395
kek:  0.2680356507386108
kek:  0.13490134498455575
kek:  0.0680244478021308
kek:  0.1375303603264797
kek:  0.3650643230174069
kek:  0.023183588454628534
kek:  0.035672915363790025
kek:  0.10024972146709388
kek:  0.29601341974085926
kek:  0.5254302538377829
kek:  0.3137062129110859
kek:  0.1466259183844727
kek:  0.5203183400922455
kek:  0.4726976210863256
kek:  0.45823369182107854
kek:  0.5032554206615316
kek:  0.008803755923793233
kek:  0.3243840242987801
kek:  0.5091105538700447
kek:  0.07762787160797302
kek:  0.37086618278829064
kek:  0.19031539916818474
kek:  0.7758638796979085
kek:  0.5525836687306196
kek:  0.1238024070621293
kek:  0.6981170496511595
kek:  0.0719478130027369
kek:  0.29042344937298026
kek:  0.0273671172517993
kek:  0.043656450204224484
kek: 